In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import sklearn as sk
from sklearn.compose import ColumnTransformer, make_column_selector

In [3]:
myData = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB544MACHINE/Final/DataRegression/train_new.csv")
myData.head()

,SalePrice,PID,Lot Frontage,Lot Area,Street,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,...,Full Bath,Half Bath,Bedroom AbvGr,TotRms AbvGrd,Gr Liv Area,Functional,Screen Porch,Pool Area,Yr Sold,Sale Type
0,159000,531363010,80.0,9605,Pave,SawyerW,1Fam,1Story,7,6,...,1,1,3,6,1218,Typ,0,0,2009,WD
1,271900,906203120,90.0,14684,Pave,SawyerW,1Fam,1Story,7,7,...,2,0,3,7,2196,Typ,0,0,2009,WD
2,137500,916176030,NaN,14375,Pave,Timber,1Fam,SLvl,6,6,...,1,0,3,7,1344,Typ,233,0,2009,COD
3,248500,528180130,48.0,6472,Pave,NridgHt,TwnhsE,1Story,9,5,...,2,0,2,6,1456,Typ,0,0,2009,WD
4,167000,528290030,61.0,9734,Pave,Gilbert,1Fam,SLvl,7,5,...,2,1,3,7,1374,Typ,0,0,2009,WD


In [6]:
def find_coefficients(pipeline, X, y, top_n=8):
    """
    Fits a pipeline with linear regression, displays top features by coefficient magnitude.

    Parameters:
    pipeline (Pipeline): Scikit-learn pipeline with preprocessing and a linear regression model.
    X (DataFrame): Predictor variables.
    y (Series): Target variable.
    top_n (int): Number of top features to display based on coefficient magnitude.

    Returns:
    data frame of top_n best features 
    """
    # Fit the pipeline to the dataset
    pipeline.fit(X, y)

    # Extract the linear model from the pipeline
    model = pipeline.named_steps['regressor']
    
    # Get feature names and coefficients
    feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
    coefficients = model.coef_

     # Create a DataFrame with feature names and coefficients
    coef_df = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': coefficients
    })
    
    # Sort the coefficients by absolute value in descending order
    coef_df['Abs_Coefficient'] = coef_df['Coefficient'].abs()
    coef_df = coef_df.sort_values(by='Abs_Coefficient', ascending=False)

    # Add weight for determining best features
    coef_df['Weight'] = [10 - i for i in range(len(coef_df))]
    
    # Get indices of features sorted by coefficient magnitude
    sorted_indices = np.argsort(np.abs(coefficients))[::-1]
    
    # Display top features by magnitude
    for i in sorted_indices[:top_n]:
        print(f"{feature_names[i]}: {coefficients[i]:.4f}")
    print("\n")
    
    return coef_df[["Feature", "Weight"]].head(10)

def estimate_rmse(pipeline, X, y):
    """
    Estimates rMSE using cross-validation for a given pipeline.

    Parameters:
    pipeline (Pipeline): Scikit-learn pipeline with preprocessing and a linear regression model.
    X (DataFrame): Predictor variables.
    y (Series): Target variable.
    cv (int): Number of cross-validation folds.

    Returns:
    float: Cross-validated rMSE
    """
    # Perform cross-validation and calculate MSE
    rmse_scores = cross_val_score(pipeline, X, y, cv=5, scoring='neg_root_mean_squared_error')
    cv_rmse = -np.mean(rmse_scores)  # Convert to positive MSE
    print(f"Estimated rMSE: {cv_rmse:,.2f}\n")
    return cv_rmse

def tune_model(model_type, pipeline, X, y, param_grid):
    """
    Tunes a regularized regression model using GridSearchCV.

    Parameters:
    model_type (str): Type of regularized regression ('lasso', 'ridge', or 'elasticnet').
    pipeline (Pipeline): Scikit-learn pipeline with preprocessing.
    X (DataFrame): Predictor variables.
    y (Series): Target variable.
    param_grid (dict): Parameter grid for hyperparameter tuning.
    cv (int): Number of cross-validation folds.

    Returns:
    GridSearchCV: Fitted GridSearchCV object with the best parameters.
    """
    # Select the model based on the input parameter
    if model_type == 'lasso':
        model = Lasso()
    elif model_type == 'ridge':
        model = Ridge()
    elif model_type == 'elasticnet':
        model = ElasticNet()
    else:
        raise ValueError("Invalid model_type. Choose from 'lasso', 'ridge', or 'elasticnet'.")

    # Update the pipeline with the selected model
    pipeline.steps[-1] = ('regressor', model)

    # Set up GridSearchCV with the pipeline and the parameter grid
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
    
    # Fit the grid search
    grid_search.fit(X, y)
    
    # Display best parameters and corresponding MSE
    print(f"Best parameters for {grid_search.best_params_}")
    
    return grid_search

In [7]:
good_cols = myData.isna().sum() < 100
myDataGood = myData.loc[:,good_cols]

# Drop other NAs
myDataDropped = myDataGood.dropna()

# Set up predictor and target variables to use for all models
X = myDataDropped.drop(columns=['SalePrice'])
y = myDataDropped['SalePrice']

# Set up ct to handle all variables in order to find best coefficients
ct = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False, handle_unknown='ignore'),make_column_selector(dtype_include=object)),
    ("standardize", StandardScaler(), make_column_selector(dtype_include=np.number))
  ],
  remainder = "passthrough"
)

In [8]:
pipeline_ols_1 = Pipeline(steps=[
    ('preprocessor', ct),
    ('regressor', LinearRegression())
])
coef1 = find_coefficients(pipeline_ols_1, X, y, top_n=5)
mse1 = estimate_rmse(pipeline_ols_1, X, y)

dummify__Street_Pave: 857268266050691840.0000
dummify__Street_Grvl: 857268266050677248.0000
dummify__House Style_2.5Fin: -407935596710997632.0000
dummify__House Style_2.5Unf: -407935596710971968.0000
dummify__House Style_2Story: -407935596710970112.0000


Estimated rMSE: 770,769,737,855,264.50



In [9]:
# Ridge regression pipeline
pipeline_ridge = Pipeline(steps=[
    ('preprocessor', ct),
    ('regressor', Ridge())
])
# Define the parameter grid for Ridge regression
ridge_param_grid = {'regressor__alpha': [0.01, 0.1, 1, 10, 100, 200]}

# Tune the Ridge model
tuned_ridge = tune_model('ridge', pipeline_ridge, X, y, ridge_param_grid)
best_ridge = tuned_ridge.best_estimator_

coef2 = find_coefficients(best_ridge, X, y, top_n=5)
mse2 = estimate_rmse(best_ridge, X, y)

Best parameters for {'regressor__alpha': 1}
dummify__Neighborhood_StoneBr: 62661.6507
dummify__Neighborhood_NridgHt: 50254.3063
dummify__Neighborhood_NoRidge: 43116.2343
standardize__Gr Liv Area: 35982.6132
dummify__Neighborhood_NWAmes: -28818.6292


Estimated rMSE: 32,858.15



In [11]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Suppress specific ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# LASSO regression pipeline
pipeline_lasso = Pipeline(steps=[
    ('preprocessor', ct),
    ('regressor', Lasso())
])
# Define the parameter grid for Ridge regression
lASSO_param_grid = {'regressor__alpha': [0.01, 0.1, 1, 10, 100]}

# Tune the Ridge model
tuned_LASSO = tune_model('lasso', pipeline_lasso, X, y, lASSO_param_grid)
best_LASSO = tuned_LASSO.best_estimator_

coef3 = find_coefficients(best_LASSO, X, y, top_n=5)
mse3 = estimate_rmse(best_LASSO, X, y)

Best parameters for {'regressor__alpha': 10}
dummify__Neighborhood_StoneBr: 67227.1187
dummify__Neighborhood_NridgHt: 53968.9127
dummify__Neighborhood_NoRidge: 47063.3223
dummify__Heating_OthW: -40654.2216
dummify__Neighborhood_GrnHill: 36944.6694


Estimated rMSE: 32,907.32



In [12]:
# Suppress specific ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# LASSO regression pipeline
pipeline_en = Pipeline(steps=[
    ('preprocessor', ct),
    ('regressor', ElasticNet())
])
# Define the parameter grid for Ridge regression
en_param_grid = {
    'regressor__alpha': [0.01, 0.1, 1, 10, 100],
    'regressor__l1_ratio': [0.1, 0.5, 0.7, 0.9, 1.0]
}

# Tune the Ridge model
tuned_en = tune_model('elasticnet', pipeline_en, X, y, en_param_grid)
best_en = tuned_en.best_estimator_

coef4 = find_coefficients(best_en, X, y, top_n=5)
mse4 = estimate_rmse(best_en, X, y)

Best parameters for {'regressor__alpha': 0.01, 'regressor__l1_ratio': 0.9}
dummify__Neighborhood_StoneBr: 59876.0752
dummify__Neighborhood_NridgHt: 49060.4679
dummify__Neighborhood_NoRidge: 42052.4162
standardize__Gr Liv Area: 35791.2531
dummify__Neighborhood_NWAmes: -28031.7491


Estimated rMSE: 32,838.22



In [13]:
coefficients = [coef1, coef2, coef3, coef4]

# Concatenate the DataFrames along rows (axis=0)
concatenated_df = pd.concat(coefficients, ignore_index=True)

# Group by 'Feature' and sum the 'Weight' values
grouped_df = concatenated_df.groupby('Feature', as_index=False)['Weight'].sum()

# Sort by the summed weights in descending order
grouped_df_sorted = grouped_df.sort_values(by='Weight', ascending=False)

# Print the sorted DataFrame
print(grouped_df_sorted)

                          Feature  Weight
15  dummify__Neighborhood_StoneBr      30
13  dummify__Neighborhood_NridgHt      27
12  dummify__Neighborhood_NoRidge      24
5     dummify__House Style_2.5Fin      21
19       standardize__Gr Liv Area      19
11   dummify__Neighborhood_NWAmes      14
1           dummify__Heating_OthW      12
18           dummify__Street_Pave      10
10  dummify__Neighborhood_GrnHill       9
17           dummify__Street_Grvl       9
6     dummify__House Style_2.5Unf       7
7     dummify__House Style_2Story       6
14  dummify__Neighborhood_OldTown       6
2     dummify__House Style_1.5Fin       5
16         dummify__Sale Type_Con       4
3     dummify__House Style_1.5Unf       4
8     dummify__House Style_SFoyer       4
9       dummify__House Style_SLvl       3
0         dummify__Bldg Type_1Fam       3
4     dummify__House Style_1Story       2
20      standardize__Overall Qual       1


In [55]:
myData_dummies = pd.get_dummies(myDataDropped, columns=['Neighborhood', 'House Style'], prefix=['Neighborhood', 'House Style'], prefix_sep='_', dtype= int)
myData_dummies["Neighborhood"] = myDataDropped["Neighborhood"]
myData_dummies["House Style"] = myDataDropped["House Style"]
myData_dummies.head()

,SalePrice,PID,Lot Area,Street,Bldg Type,Overall Qual,Overall Cond,Year Built,Roof Style,Heating,...,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Neighborhood,House Style
0,159000,531363010,9605,Pave,1Fam,7,6,2007,Gable,GasA,...,0,0,1,0,0,0,0,0,SawyerW,1Story
1,271900,906203120,14684,Pave,1Fam,7,7,1990,Hip,GasA,...,0,0,1,0,0,0,0,0,SawyerW,1Story
2,137500,916176030,14375,Pave,1Fam,6,6,1958,Gable,GasA,...,0,0,0,0,0,0,0,1,Timber,SLvl
3,248500,528180130,6472,Pave,TwnhsE,9,5,2008,Hip,GasA,...,0,0,1,0,0,0,0,0,NridgHt,1Story
4,167000,528290030,9734,Pave,1Fam,7,5,2004,Gable,GasA,...,0,0,0,0,0,0,0,1,Gilbert,SLvl


In [67]:
warnings.filterwarnings("ignore", category=RuntimeWarning)
from sklearn.impute import SimpleImputer

# make predictors based on top coefficients
X_final = myData_dummies[["Neighborhood_StoneBr","Neighborhood_NridgHt", "Neighborhood_NoRidge", "Neighborhood", "House Style_2.5Fin", "House Style", "Gr Liv Area", "Heating", "Street", "Sale Type", "Bldg Type", "Overall Qual"]]


ct_final = ColumnTransformer(
    [
        ("standardize", StandardScaler(), ["Gr Liv Area", "Overall Qual"]),
        ("quality_interaction", PolynomialFeatures(degree=2, interaction_only=True, include_bias=False), ["Gr Liv Area", "Overall Qual"]),
        ("liv_interaction", PolynomialFeatures(degree=2, interaction_only=True, include_bias=False), ["Gr Liv Area", "Neighborhood_StoneBr"]),
        ("liv_interaction2", PolynomialFeatures(degree=2, interaction_only=True, include_bias=False), ["Gr Liv Area", "Neighborhood_NridgHt"]),
        ("liv_interaction3", PolynomialFeatures(degree=2, interaction_only=True, include_bias=False), ["Gr Liv Area", "Neighborhood_NoRidge"]),
        ("poly_neigh1", PolynomialFeatures(degree= 3), ["Neighborhood_StoneBr"]),
        ("poly_neigh2", PolynomialFeatures(degree= 3), ["Neighborhood_NridgHt"]),
        ("poly_neigh3", PolynomialFeatures(degree= 3), ["Neighborhood_NoRidge"]),
        ("poly_sty1", PolynomialFeatures(degree= 3), ["House Style_2.5Fin"]),
        ("poly_qual", PolynomialFeatures(degree= 3), ["Overall Qual"]),
        ("poly_liv", PolynomialFeatures(degree= 3), ["Gr Liv Area"]),
        ("dummify", OneHotEncoder(handle_unknown="ignore", sparse_output=False), ["House Style", "Neighborhood", "Heating", "Street", "Sale Type", "Bldg Type"]),
    ],
    remainder="drop"
)


pipeline_Final = Pipeline(steps=[
    ('preprocessor', ct_final),
    ('regressor', ElasticNet(max_iter=10000))
])

param_grid = {
    # Define the parameter grid for degrees
    #'preprocessor__poly_neigh1__degree': np.arange(1,4),
    #'preprocessor__poly_neigh2__degree': np.arange(1,4),
    #'preprocessor__poly_neigh3__degree': np.arange(1,4),
    #'preprocessor__poly_sty1__degree': np.arange(1,4),
    # Define the parameter grid for alpha and lambda
    'regressor__alpha': [.01,],
    'regressor__l1_ratio': [0.9]
}

gs_final = GridSearchCV(pipeline_Final, param_grid, cv=5, scoring='neg_root_mean_squared_error')
gs_final_fitted = gs_final.fit(X_final, y)

In [75]:
best_params = gs_final_fitted.best_params_

# Format and display the best parameters cleanly
formatted_params = {
    key: value.item() if isinstance(value, np.int64) else value
    for key, value in best_params.items()
}

# Print with better formatting
print("Best parameters:")
for param, value in formatted_params.items():
    print(f"{param}: {value}")
best_en_score = -gs_final.best_score_
print(f"\nEstimated rMSE: {float(best_en_score):,.2f}")

Best parameters:
regressor__alpha: 0.01
regressor__l1_ratio: 0.9

Estimated rMSE: 29,180.35


In [76]:
myScoreData = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB544MACHINE/Final/DataRegression/test_new.csv")

myScoreData_dummies = pd.get_dummies(myScoreData, columns=['Neighborhood', 'House Style'], prefix=['Neighborhood', 'House Style'], prefix_sep='_', dtype= int)
myScoreData_dummies["Neighborhood"] = myScoreData["Neighborhood"]
myScoreData_dummies["House Style"] = myScoreData["House Style"]
myScoreData_dummies.head()


X_test = myScoreData_dummies[["Neighborhood_StoneBr","Neighborhood_NridgHt", "Neighborhood_NoRidge", "Neighborhood", "House Style_2.5Fin", "House Style", "Gr Liv Area", "Heating", "Street", "Sale Type", "Bldg Type", "Overall Qual"]]

# Use the best estimator from the GridSearchCV to predict
y_pred = gs_final.best_estimator_.predict(X_test)

# Create a new DataFrame to ensure it's a clean copy
finaldf = myScoreData[["PID"]].copy()

# Add the predictions as a new column
finaldf["SalePrice"] = y_pred

# Check the result
finaldf.head()


,PID,SalePrice
0,907135180,137468.558544
1,528181040,212324.678536
2,528175010,210998.257157
3,531379030,184796.739699
4,923275090,133385.488096


In [77]:
# Save the finaldf DataFrame to a CSV file
finaldf.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB544MACHINE/Final/DataRegression/predicted_sale_prices3.csv", index=False)
